# What is this?

This library started to improve the current [pylutron_caseta](https://github.com/gurumitts/pylutron-caseta) package, but I got a bit nuts enumerating all the messages, so I renamed it since it would be a very significant pull request.

That said, I'm happy to work it back in, if there is a desire.

## The Approach

I reverse engineered the protocol primarily using my current RA3 processor, capturing network traffic using a MITM SSL proxy. I generated many actions from a couple different systems to record how they worked.

## The state of things

I'm confident in the API implementation, that it is complete in the currently available features of the platform. The models need a bit more work, but subscriptions and commands do currently work. I'm working out some bugs in the callbacks.

## How it's supposed to work

The way I've observed other systems interacting with RA3 the startup flow goes like this:

1. Connect over TLS (cert is signed using the known chain with a root Lutron CA)
2. Authenticate
   1. (Option A) Send an authentication message using a username and password. When authentication is performed in this manner, the session runs at a reduced privilege level that offers "monitor", and "control". 
   2. (Option B) Authenticate with x509 cert that has been signed from the RA3 CA using the established process (similar to one used by Caseta).
3. Subscribe to all zones and areas for status changes. The response to these requests enumerates all zones and areas. The idea is to create models for each respective item, but there's some flaw in the logic there currently.

### Code block 1

This block shows basic schema usage to dump a message header using enums and string values.

In [ ]:
from pylutron_leap.api.message import ResponseStatus, LeapMessageHeader, LeapMessage, LeapDirectives
from pylutron_leap.api.enum import CommuniqueType, MessageBodyTypeEnum



r = ResponseStatus.from_str("200 OK")


_obj: LeapMessageHeader = LeapMessageHeader(
    Url="/not/a/real/endpoint",
    StatusCode=ResponseStatus.from_str("200 OK"),
    MessageBodyType=MessageBodyTypeEnum.OneAreaStatus,
    Directives=LeapDirectives(SuppressMessageBody=False),
)

LeapMessageHeader.Schema().dump(_obj)

# _val = LeapMessage(CommuniqueType=CommuniqueType.ReadRequest, Header=LeapHeader(Url="/server/status/ping"))
# LeapMessage.Schema().dump(_val)

### Code block 2

This block shows basic schema usage for an entire message type.

In [ ]:

from pylutron_leap.api.emergency import EmergencyStatus, LeapEmergencyBody
from pylutron_leap.api.enum import (
    CommuniqueType,
    EmergencyStateEnum,
    MessageBodyTypeEnum,
)
from pylutron_leap.api.message import LeapMessage, LeapMessageHeader

_obj = LeapMessage(
        CommuniqueType=CommuniqueType.UpdateRequest,
        Header=LeapMessageHeader(Url="/emergency/flash/status"),
        Body=LeapEmergencyBody(
            EmergencyStatus=EmergencyStatus(ActiveState=EmergencyStateEnum.Active)
        ),
    )

LeapMessage.Schema().dump(_obj)


In this cell, I'm playing with the session trying to get the "normal" process flow to work. Some of the APIs are not available when authenticating with username/password. Ideally, we can find work arounds for those.

In [ ]:
import asyncio
import json
import os
from logging import getLogger

from dotenv import load_dotenv

from pylutron_leap.api.enum import (
    CommuniqueType,
    ContextTypeEnum,
    FanSpeedType,
    MessageBodyTypeEnum,
)
from pylutron_leap.api.login import LeapLoginBody, LoginBody
from pylutron_leap.api.message import LeapMessage, LeapMessageHeader
from pylutron_leap.models.area import Area
from pylutron_leap.models.fan import FanModel
from pylutron_leap.models.zone import Zone
from pylutron_leap.session import LeapSession

logger = getLogger()

asyncio.get_running_loop().set_debug(True)

# Load connection parameters from `.env` file
load_dotenv()
lutron_host=os.getenv('LUTRON_HOST')
lutron_user=os.getenv('LUTRON_USER')
lutron_pass=os.getenv('LUTRON_PASS')

_config = {
    "host": lutron_host,
    "port": 8081,
    "ca_chain": "caseta-bridge.crt",
    "keyfile": "caseta.key",
    "certfile": "caseta.crt",
    "username": lutron_user,
    "password": lutron_pass,
    "verify_tls": True
}
session = LeapSession(**_config)
# session.close()

await session.connect()
logger.debug("Connected...presumably")

logger.debug(f"Logged in: {session.logged_in}")

await session._login_completed

await asyncio.sleep(3)

logger.debug("=================================================================")
# _areas = await Area.get_areas(session)
logger.debug(list(session.areas))

logger.debug("=================================================================")

# _areas = await Area.get_areas(session)
logger.debug(list(session.zones))

logger.debug("=================================================================")

logger.debug(list(session.devices))


logger.debug("=================================================================")

# logger.debug(Zone.zones)

# json.dumps(LeapMessage.Schema().dump(_msg))

# This is 15 minutes, which is a bit much, but you need to sleep in the notebook to receive the occupied events for the area
await asyncio.sleep(900) 
